In [ ]:
import numpy as np  
import pandas as pd  

import matplotlib.pyplot as plt # For plots
from sklearn.model_selection import train_test_split
import seaborn as sns
import pickle
import datetime
import re
pd.set_option('display.max_columns', None)


In [ ]:
def column_summary(col,dataframe):
    # column summary - pass col name as string + dataframe. Receive sorted occurence count and %
    # TODO nan handling - atm not showing
    pd.set_option('display.max_rows', None)

    counts = dataframe[col].value_counts()
    percs = dataframe[col].value_counts(normalize=True).mul(100).round(3).astype(str) + '%'
    return pd.concat([counts,percs], axis=1, keys=['count', 'percentage'])


def analyse_column_list(col_lst, df,show=True):
    if show == False:
        return None
    else:
        col_lst1 = column_name_adjuster(col_lst, df)
        for column in col_lst1:
            print("\n*********",'\033[1m' + column + '\033[0m')
            display(column_summary(column,df))

In [ ]:
df_appr_full = pd.read_pickle("./IN_dfs/df_appr_full_raw.pkl")
df_comp_full = pd.read_pickle("./IN_dfs/df_comp_full_raw.pkl")
print("Appriasal Shape", df_appr_full.shape)
print("Comp Shape", df_comp_full.shape)
pd.set_option('display.max_rows', 1000)

In [ ]:
df_comp_full.columns

In [ ]:
special_columns = [ 'DESIGNSTYLE_SUBJ', 'HEATINGCOOLING_SUBJ', 'ENERGYEFF_SUBJ',
       'GARAGECARPORT_SUBJ', 'PORCHPATIODECK_SUBJ' ]

In [ ]:
pd.set_option('display.max_rows', 1000)
col = pd.concat([df_appr_full[ 'DESIGNSTYLE_SUBJ'],df_comp_full[ 'DESIGNSTYLE']])
col=col.str.lower()
df_temp=pd.DataFrame()
df_temp["DESIGN_STYLE_ORIG"] = col.copy()
df_temp["no_comma"] = df_temp["DESIGN_STYLE_ORIG"].str.replace(',', '.')
df_temp["no_comma"] = df_temp["DESIGN_STYLE_ORIG"].str.replace(' ', '.')
df_temp['extracted_Number'] = df_temp['no_comma'].str.replace('([A-Za-z]+)', '',regex=True)


In [ ]:
df_nostory = df_temp[~df_temp['extracted_Number'].str.contains(r'\d+')]# 976083 c+a records with no story number - remove
column_summary("DESIGN_STYLE_ORIG",df_nostory)

In [ ]:
column_summary("DESIGN_STYLE_ORIG",df_nostory)

In [ ]:

#clean up the extracted_number field

lst = df_temp['extracted_Number'].unique()
#print(len(lst))
import re
lst_num = []
for no in lst:
    #print(lst)
    #print (no)
    try:
        test = float(no)
        lst_num.append(test)
    except Exception as e: 
        #print(no)
        #x = no.lstrip()
        x = no.lstrip('0')
        #print(x)
        temp_list = re.findall(r'[+-]?([1-9]\d*(\.\d*[1-9])?|0\.\d*[1-9]+)|\d+(\.\d*[1-9])?', x)#look for numbers
        if len(temp_list)==0:
            lst_num.append(np.nan)
        else:
            #print(temp_list)
            try:
                lst_num.append(float(temp_list[0][0]))#take first number
            except Exception as e: 
                try: 
                    a = temp_list[0][0]
                    a = a.replace('.', '')
                    a = a.replace(',', '')
                    lst_num.append(float(a))#take second number
                except Exception as ee:
                    lst_num.append(np.nan)

site_dict = dict(zip(lst, lst_num))

df_temp["Number_of_stories_no_imputation"] = df_temp["extracted_Number"].map(site_dict.get) #QUICKEST!!!!

In [ ]:
df_temp["Number_of_stories_no_imputation"] = abs(df_temp["Number_of_stories_no_imputation"])

df_temp['Number_of_stories_no_imputation_divided'] = np.round(np.where(df_temp['Number_of_stories_no_imputation'] >=10, 
                                                                  df_temp['Number_of_stories_no_imputation']/10,
                                                                       df_temp['Number_of_stories_no_imputation']), 1)
df_temp['Number_of_stories_no_imputation_divided'] = np.round(np.where(df_temp['Number_of_stories_no_imputation_divided'] >=10, 
                                                                  df_temp['Number_of_stories_no_imputation_divided']/10,
                                                                       df_temp['Number_of_stories_no_imputation_divided']), 1)

df_temp['Number_of_stories_no_imputation_divided'] = np.round(np.where(df_temp['Number_of_stories_no_imputation_divided'] >=6, 
                                                                  np.nan,
                                                                       df_temp['Number_of_stories_no_imputation_divided']), 1)


dict_fix_floors = {0.4:2, 0.8:np.nan, 5.2: np.nan, 0.0: np.nan, 0.1:1.0, 0.2:2.0, 0.3:3, 0.7:np.nan,}

df_temp["Number_of_stories_no_imputation_divided"] = df_temp["Number_of_stories_no_imputation_divided"].replace(dict_fix_floors)

In [ ]:
column_summary("Number_of_stories_no_imputation_divided",df_temp)

In [ ]:
df_temp = df_temp.drop(columns = ["no_comma","extracted_Number","Number_of_stories_no_imputation"])
df_temp = df_temp.rename(columns={'Number_of_stories_no_imputation_divided': 'Number_of_stories_no_imputation'})
print(df_temp[df_temp["Number_of_stories_no_imputation"].isnull()].shape)# 976k of rows with no stories
df_temp["Number_of_stories_impute_to1"] = df_temp["Number_of_stories_no_imputation"].fillna(1.0)

In [ ]:
%%time
# Write a function, using simple if... elif syntax 
def _conditions1(row):    
    if row["DESIGN_STYLE_ORIG"].startswith("dt"): Attachment_type = "DT"
    elif row["DESIGN_STYLE_ORIG"].startswith("sd"): Attachment_type = "SD"
    elif row["DESIGN_STYLE_ORIG"].startswith("at"): Attachment_type = "AT"
    elif row["DESIGN_STYLE_ORIG"].find("ranch")>-1: Attachment_type = "DT"
    elif row["DESIGN_STYLE_ORIG"].find("trad")>-1: Attachment_type = "DT"
    elif row["DESIGN_STYLE_ORIG"].find("bunga")>-1: Attachment_type = "DT"
    elif row["DESIGN_STYLE_ORIG"].find("spa")>-1: Attachment_type = "DT"
    elif row["DESIGN_STYLE_ORIG"].find("col")>-1: Attachment_type = "DT"
    elif row["DESIGN_STYLE_ORIG"].find("uth")>-1: Attachment_type = "DT"
    elif row["DESIGN_STYLE_ORIG"].find("sth")>-1: Attachment_type = "DT"
    elif row["DESIGN_STYLE_ORIG"].find("end")>-1: Attachment_type = "SD"
    elif row["DESIGN_STYLE_ORIG"].find("town")>-1: Attachment_type = "AT"
    elif row["DESIGN_STYLE_ORIG"].find("twn")>-1: Attachment_type = "AT"
    elif row["DESIGN_STYLE_ORIG"].find("th")>-1: Attachment_type = "AT"
    elif row["DESIGN_STYLE_ORIG"].find("row")>-1: Attachment_type = "AT"
    elif row["DESIGN_STYLE_ORIG"].find("cond")>-1: Attachment_type = "AT"

    else: Attachment_type = "DT"
        
    return Attachment_type
# Create a new column based on the function
df_temp["Attachment_type"] = df_temp.apply(_conditions1, axis=1)
df_temp.head(25)

In [ ]:
df_temp = pd.concat([df_temp, pd.get_dummies(df_temp["Attachment_type"])],axis=1)

In [ ]:
%%time
df_temp['DS_Bungalow'] = np.where(df_temp['DESIGN_STYLE_ORIG'].str.contains("|".join(["1story","1sty","1 level","one","sngl","1-story","1.5","single-level","one-story","bung","bgl","bng","1 story",
                                                                                       "single level","one story","singlelvl","single lvl"])), 1, 0)
df_temp['DS_Cabin'] = np.where(df_temp['DESIGN_STYLE_ORIG'].str.contains("|".join(["cabin","log","chalet","alpine"])), 1, 0)
df_temp['DS_Classical'] = np.where(df_temp['DESIGN_STYLE_ORIG'].str.contains("|".join(["clas"])), 1, 0)
df_temp['DS_Colonial'] = np.where(df_temp['DESIGN_STYLE_ORIG'].str.contains("|".join(["colon","a-col","col"])), 1, 0)
df_temp['DS_Cottage'] = np.where(df_temp['DESIGN_STYLE_ORIG'].str.contains("|".join(["cott"])), 1, 0)
df_temp['DS_Contemp'] = np.where(df_temp['DESIGN_STYLE_ORIG'].str.contains("|".join(["temp","cont", "ctmp","modern","cntm","custom","cntp","ctmpr"])), 1, 0)
df_temp['DS_Conventional'] = np.where(df_temp['DESIGN_STYLE_ORIG'].str.contains("|".join(["conv"])), 1, 0)
df_temp['DS_CapeCod'] = np.where(df_temp['DESIGN_STYLE_ORIG'].str.contains("|".join(["cape", "cod"])), 1, 0)
df_temp['DS_Craftsman'] = np.where(df_temp['DESIGN_STYLE_ORIG'].str.contains("|".join(["craft","crf"])), 1, 0)
df_temp['DS_Duplex'] = np.where(df_temp['DESIGN_STYLE_ORIG'].str.contains("|".join(["dupl"])), 1, 0)
df_temp['DS_English'] = np.where(df_temp['DESIGN_STYLE_ORIG'].str.contains("|".join(["engl","regency","queen","anne"])), 1, 0)
df_temp['DS_French'] = np.where(df_temp['DESIGN_STYLE_ORIG'].str.contains("|".join(["french"])), 1, 0)
df_temp['DS_Georgian'] = np.where(df_temp['DESIGN_STYLE_ORIG'].str.contains("|".join(["georg","geor","grg"])), 1, 0)
df_temp['DS_Medit'] = np.where(df_temp['DESIGN_STYLE_ORIG'].str.contains("|".join(["med","mdtrn","mdrn"])), 1, 0)
df_temp['DS_MidCentury'] = np.where(df_temp['DESIGN_STYLE_ORIG'].str.contains("|".join(["mid-","century","mid ","midc"])), 1, 0)
df_temp['DS_Mountain'] = np.where(df_temp['DESIGN_STYLE_ORIG'].str.contains("|".join(["mtn","mount","mdrn"])), 1, 0)
df_temp['DS_NeoEclect'] = np.where(df_temp['DESIGN_STYLE_ORIG'].str.contains("|".join(["ecl","neoe","neoclctc","neo e"])), 1, 0)
df_temp['DS_NewAmerican'] = np.where(df_temp['DESIGN_STYLE_ORIG'].str.contains("|".join(["amer","newame","amcn","new am","newam"])), 1, 0)
df_temp['DS_Patio'] = np.where(df_temp['DESIGN_STYLE_ORIG'].str.contains("|".join(["patio","tract"])), 1, 0)
df_temp['DS_Ranch'] = np.where(df_temp['DESIGN_STYLE_ORIG'].str.contains("|".join(["ranch","rnch","rnh","farm","rch","calif","rh","frm"])), 1, 0)
df_temp['DS_Rambler'] = np.where(df_temp['DESIGN_STYLE_ORIG'].str.contains("|".join(["ramb","rmblr"])), 1, 0)
df_temp['DS_SantaBarb'] = np.where(df_temp['DESIGN_STYLE_ORIG'].str.contains("|".join(["barb","brb"])), 1, 0)
df_temp['DS_Spanish'] = np.where(df_temp['DESIGN_STYLE_ORIG'].str.contains("|".join(["span","villa","santa fe","fe","spn","spain"])), 1, 0)
df_temp['DS_SplitLevel'] = np.where(df_temp['DESIGN_STYLE_ORIG'].str.contains("|".join(["split","multi","bi-level"])), 1, 0)
df_temp['DS_SWest'] = np.where(df_temp['DESIGN_STYLE_ORIG'].str.contains("|".join(["south", "sth", "sw", "west","wst"])), 1, 0)
df_temp['DS_Territorial'] = np.where(df_temp['DESIGN_STYLE_ORIG'].str.contains("|".join(["terri"])), 1, 0)

df_temp['DS_Townhouse'] = np.where(df_temp['DESIGN_STYLE_ORIG'].str.contains("|".join(["wth","twh","t-house","tnh",
                        "row","twn","town","thouse","twnh",
                        "thse","rowth","rowhouse",
                    "th/int","intth","midth","endth","th/","/th","t/h",";th","-th"])), 1, 0) #,"th/","/th"

df_temp['DS_Trad'] = np.where(df_temp['DESIGN_STYLE_ORIG'].str.contains("|".join(["trad","trd"])), 1, 0)
df_temp['DS_Tudor'] = np.where(df_temp['DESIGN_STYLE_ORIG'].str.contains("|".join(["tudo"])), 1, 0)
df_temp['DS_Tuscan'] = np.where(df_temp['DESIGN_STYLE_ORIG'].str.contains("|".join(["tusc","ital"])), 1, 0)
df_temp['DS_TwoStory'] = np.where(df_temp['DESIGN_STYLE_ORIG'].str.contains("|".join(["sq","square","2sty","2 sty","2story","splt","bilevel","two","2 story","2-story", "two story", "bi level","splt lvl","tri-level"])), 1, 0)
df_temp['DS_Victorian'] = np.where(df_temp['DESIGN_STYLE_ORIG'].str.contains("|".join(["vict"])), 1, 0)
df_temp['_DS_SUM'] = df_temp[[col for col in df_temp.columns if col.startswith('DS_')]].sum(axis=1)


In [ ]:
df_temp = df_temp.drop(columns = ["Attachment_type"])

In [ ]:
df_temp.to_pickle("./OUT_dfs/df_temp.pkl")

In [ ]:
#print Deliberate stop here to avoid memory error

# Load appraisal dataframe to do the merge

In [ ]:
import numpy as np  
import pandas as pd 

In [ ]:
df_temp = pd.read_pickle("./OUT_dfs/df_temp.pkl")
df_appr_nona = pd.read_pickle("./OUT_dfs/df_appr_full_processed_nona_no_outliers.pkl")
df_appr_full = pd.read_pickle("./IN_dfs/df_appr_full_raw.pkl")

In [ ]:
# apply radians to long and lat for later use
df_appr_nona[['APPRLATITUDE', 'APPRLONGITUDE']]=df_appr_nona[['APPRLATITUDE', 'APPRLONGITUDE']].apply(np.radians,axis=0)

 

In [ ]:
df_a = df_appr_nona.copy()


In [ ]:
# rename columns in full appr dataframe to get ready for merge
new_names = {'DESIGNSTYLE_SUBJ':'DESIGNSTYLE', 'HEATINGCOOLING_SUBJ':'HEATINGCOOLING', 'ENERGYEFF_SUBJ':'ENERGYEFF', 
             'GARAGECARPORT_SUBJ':'GARAGECARPORT',"PORCHPATIODECK_SUBJ":"PORCHPATIODECK"}
df_appr_full = df_appr_full.rename(index=str, columns=new_names)

In [ ]:
df_appr_full[['DESIGNSTYLE', 'HEATINGCOOLING', 'ENERGYEFF',
       'GARAGECARPORT', 'PORCHPATIODECK' ]].head()

In [ ]:
# Merge df_a and df_c with full ones to bring back the above 4 features

df_a_new = pd.merge(df_a,df_appr_full[['SUBJ_APPR_ID','DESIGNSTYLE', 'HEATINGCOOLING', 'ENERGYEFF',
       'GARAGECARPORT', 'PORCHPATIODECK' ]], how="left",left_on='SUBJ_APPR_ID', right_on='SUBJ_APPR_ID')

In [ ]:
df_a_new.to_pickle("./OUT_dfs/df_a_new_ignore.pkl")

In [ ]:
#print Deliberate stop here to avoid memory error

# next merge

In [ ]:
import numpy as np  
import pandas as pd 

In [ ]:

df_a_new = pd.read_pickle("./OUT_dfs/df_a_new_ignore.pkl")

In [ ]:

df_a_new["DESIGNSTYLE"]=df_a_new["DESIGNSTYLE"].astype(str)

In [ ]:
#df_temp["DESIGNSTYLE"].dtypes
df_a_new["DESIGNSTYLE"].dtypes

In [ ]:
df_temp = pd.read_pickle("./OUT_dfs/df_temp.pkl")
df_temp.rename(columns={'DESIGN_STYLE_ORIG':'DESIGNSTYLE'}, inplace=True)
df_temp["DESIGNSTYLE"]=df_temp["DESIGNSTYLE"].astype(str)
dict1 = df_temp.set_index("DESIGNSTYLE").apply(list, 1).to_dict()

In [ ]:
pd.set_option('display.max_columns', 1000)
df_a_newzzzz = df_a_new.copy()
df_a_newzzzz["DSTEST"] = df_a_new["DESIGNSTYLE"].str.lower().map(dict1.get)
df_a_newzzzz[df_temp.columns[1:]] = pd.DataFrame(df_a_newzzzz.DSTEST.tolist(), index= df_a_newzzzz.index)

df_a_newzzzz = df_a_newzzzz.drop(columns = ["DSTEST"])
#df_a_newzzzz.tail(25)

In [ ]:
df_a_newzzzz.to_pickle("./OUT_dfs/df_a1.pkl")

# Load Comp dataframe to merge

In [ ]:
import numpy as np  
import pandas as pd 

In [ ]:
df_comp_full = pd.read_pickle("./IN_dfs/df_comp_full_raw.pkl")
uniqueComps = pd.read_pickle("./OUT_dfs/uniqueComps.pkl")
uniqueComps[['APPRLATITUDE', 'APPRLONGITUDE']]=uniqueComps[['APPRLATITUDE', 'APPRLONGITUDE']].apply(np.radians,axis=0)
df_c = uniqueComps.copy()
#df_c.head()

In [ ]:
df_c_new = pd.merge(df_c,df_comp_full[['SUBJ_APPR_ID',"COMPNUM",'DESIGNSTYLE', 'HEATINGCOOLING', 'ENERGYEFF',
       'GARAGECARPORT', 'PORCHPATIODECK' ]], how="left",left_on=['SUBJ_APPR_ID',"COMPNUM"], right_on=['SUBJ_APPR_ID',"COMPNUM"])

In [ ]:
df_c_new["DESIGNSTYLE"]=df_c_new["DESIGNSTYLE"].astype(str)

In [ ]:
pd.set_option('display.max_columns', 1000)
df_a_newzzzz = df_c_new.copy()
df_a_newzzzz["DSTEST"] = df_c_new["DESIGNSTYLE"].str.lower().map(dict1.get)
df_a_newzzzz[df_temp.columns[1:]] = pd.DataFrame(df_a_newzzzz.DSTEST.tolist(), index= df_a_newzzzz.index)

df_a_newzzzz = df_a_newzzzz.drop(columns = ["DSTEST"])


In [ ]:
df_a_newzzzz.to_pickle("./OUT_dfs/df_c1.pkl")